## Master Thesis Data Collection

Alessio Gattoni 

alessio.gattoni@student.unisg.ch

In [ ]:
#import all the necessary packages to scrape the data

from gnews import GNews 
from tqdm import tqdm
import pandas as pd
from datetime import datetime
from newspaper import Article
import time

In [ ]:
#sample the artlicle links using the GNews package

# loop through the various news sources and get the articles
# the gnews package is facing some issues lately, the number of articles returned is not as extensive as it used to be

df = [] 
for year in tqdm(range(2020, 2024)):
    for month in range(1, 13):
        if month == 2:
            last = 28
        elif month in [4, 6, 9, 11]:
            last = 30
        else:
            last = 31
        google_news = GNews(start_date=(year, month, 1), end_date=(year, month, 15))
        response = GNews.get_news_by_site(google_news, site ="nzz.ch/wirtschaft")
        print(len(response))
        for r in response:
            df.append(r)  
        google_news = GNews(start_date=(year, month, 16), end_date=(year, month, last))
        response = GNews.get_news_by_site(google_news, site ="nzz.ch/wirtschaft")
        print(len(response))
        for r in response:
            df.append(r)

In [ ]:
len(df)

In [ ]:
# organize the dataframe

url_data = []
for i in range(0,len(df)):
    url_data.append(df[i]["url"])
    
title_data = []
for i in range(0,len(df)):
    title_data.append(df[i]["title"])
    
published_date_data = []
for i in range(0,len(df)):
    date_string = df[i]["published date"]
    date_format = "%a, %d %b %Y %H:%M:%S %Z"
    date_object = datetime.strptime(date_string, date_format)
    new_date_format = "%d/%m/%Y"
    new_date = date_object.strftime(new_date_format)
    date_object = datetime.strptime(new_date, new_date_format)
    published_date_data.append(date_object)

publisher_data = []
for i in range(0,len(df)):
    publisher_data.append(df[i]["publisher"]["title"])

df = pd.DataFrame({"Title": title_data, "URL" : url_data, "Date" : published_date_data, "Publisher" : publisher_data})

In [ ]:
df

In [ ]:
# Extract whole text using the newspaper3k package

text_list = []
for i in tqdm(range(0, len(df))):
    article = Article(df["URL"][i])
    try:
        article.download()
        article.parse()
        text = article.text
        text_list.append(text)
    except:
        text_list.append("not_found")


df["Text"] = text_list

In [ ]:
df.to_excel("nzz_v1.xlsx", index=False) #save the data in excel file

#merge tha various sources in one dataframe in a separate file